In [ ]:
!pip install llama-index --quiet

In [ ]:
from llama_index.llms import AzureOpenAI
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext, get_response_synthesizer, set_global_service_context
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.embeddings import HuggingFaceEmbedding
import os
from google.colab import userdata

# LLM
Nesse caso, utlizando o modelo [GPT 3.5 com janela de contexto de 16 mil tokens](https://platform.openai.com/docs/models/gpt-3-5). Mais especificamente, utlizando um com [envolcro do serviço da Azure](https://docs.llamaindex.ai/en/stable/api_reference/llms/azure_openai.html) feito pelo framework LLamaIndex.

In [ ]:
api_key = userdata.get('OPENAI_API_KEY')
azure_endpoint = userdata.get('OPENAI_API_BASE')
api_version = "2023-03-15-preview"

llm = AzureOpenAI(
    model="gpt-35-turbo-16k",
    deployment_name="gpt-16k",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

# Embed Model
Os modelos de *embed* são utilizados para representar documentos e consultas em uma forma numérica que captura a semântica do texto. Esses *embeddings* são empregados durante a construção do índice e ao realizar consultas usando o mecanismo de consulta. O modelo de *embeddings* é essencial, pois permite ao sistema entender e comparar o significado de diferentes textos. Neste contexto, estamos utilizando o modelo [e5]( https://huggingface.co/intfloat/multilingual-e5-smal), que é considerado o melhor modelo multilíngue de código aberto para a tarefa de recuperação de informação semântica, de acordo com o ranking [MTEB](https://huggingface.co/spaces/mteb/leaderboard), que mede quão bons são os modelos de para a tarefa de busca. Além disso, utilizamos o invólucro do LlamaIndex em combinação com o hub HuggingFace para essa finalidade.


In [ ]:
embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-small")

# Setup
O código abaixo está definindo um contexto de serviço global no LlamaIndex. O ServiceContext é uma classe que agrupa vários componentes que são usados em conjunto para realizar tarefas no LlamaIndex. Isso inclui o modelo de linguagem grande (LLM), o modelo de *embeddings*, e outros componentes. A função from_defaults é usada para criar um ServiceContext com valores padrão para todos os componentes, mas permite que você substitua alguns deles, como o llm e o embed_model neste caso.

In [ ]:
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model
)
set_global_service_context(service_context)

# Ingestão de dados
Como citado, estamos utilizando dados referentes a pós-graduação do Instituto de Computação da Unicamp. Mais especificamente, sobre a [pagina de defesa do mestrado e doutorado](https://ic.unicamp.br/pos-graduacao/informacao-para-docentes-e-pos-graduandos/vida-academica-da-pos-graduacao/procedimentos-e-formularios-para-defesa-de-mestrado-e-doutorado/).

O framework utlizado permite a [ingestão de dados diretamente de um diretório](https://gpt-index.readthedocs.io/en/latest/examples/data_connectors/simple_directory_reader.html), nesse caso /data. Com a designação inicial, é possível apontar o arquivo que sera utlizado para a criação da base de *embeddings*. A partir dele, é possível criar um [index](https://docs.llamaindex.ai/en/stable/api_reference/indices/vector_store.html), que consegue indexar as informações contidas dento do arquivo.




In [ ]:
documents = SimpleDirectoryReader(
    input_files=["./data/ic.txt"]
).load_data()
index = VectorStoreIndex.from_documents(documents)


## Configuração do *Retriever*
O [VectorIndexRetriever]( https://docs.llamaindex.ai/en/stable/api_reference/query/retrievers/vector_store.html) no LlamaIndex é um componente que usa um modelo de linguagem grande (LLM) para definir automaticamente os parâmetros de consulta do armazenamento de vetores. Ele recebe dois parâmetros principais: index e similarity_top_k. O parâmetro index é o índice do armazenamento de vetores que o Retriever irá consultar. O parâmetro similarity_top_k determina o número de resultados mais semelhantes a serem retornados pela consulta. Em outras palavras, o VectorIndexRetriever é uma ferramenta que ajuda a encontrar as informações mais relevantes em um conjunto de dados, com base na semelhança com a consulta fornecida.



In [ ]:
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=2,
)

## Escolher tipo de resposta gerada pela LLM
O [get_response_synthesizer](https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/response_modes.html) no LlamaIndex é uma função que gera uma resposta a partir de um Modelo de Linguagem Grande (LLM), usando uma consulta do usuário e um conjunto de trechos de texto. O resultado é um objeto de resposta. O método para fazer isso pode assumir muitas formas, desde algo simples como iterar sobre trechos de texto, até algo complexo como construir uma árvore. A ideia principal é simplificar o processo de gerar uma resposta usando um LLM em seus dados.



In [ ]:
response_synthesizer = get_response_synthesizer(
    response_mode="simple_summarize",
)

## Motor de busca e criação de resposta
O [RetrieverQueryEngine](https://gpt-index.readthedocs.io/en/latest/examples/query_engine/CustomRetrievers.html) é um componente do LlamaIndex que permite combinar uma consulta em linguagem natural com a recuperação de uma resposta relevante. Ele depende de um "retriever" para recuperar informações de um índice, que pode ser um arquivo de texto, e de um "response_synthesizer" como parâmetro, que ajusta a forma como a resposta é criada. Este componente principal é responsável por processar a consulta do usuário e gerar uma resposta apropriada com base nas informações recuperadas pelo retriever




In [ ]:
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

## Question 1

In [ ]:
query = "Quantos membros deve ter uma banca de defesa de dissertação?"
query_engine = index.as_query_engine()
answer = query_engine.query(query)

print(answer.get_formatted_sources())
print("query was:", query)
print("answer was:", answer)

> Source (Doc id: 25bffb9a-7c99-40ea-8bb1-c885fe24f1ea): O processo de defesa de Mestrado e de Doutorado é a conclusão da trajetória do aluno no curso.
To...

> Source (Doc id: fa1edc89-b6a4-4d59-bb85-950f456ca710): Este arquivo deverá ser impresso, assinado por você e seu/sua orientador(a) e entregue à Secretar...
query was: Quantos membros deve ter uma banca de defesa de dissertação?
answer was: Uma banca de defesa de dissertação deve ter pelo menos três membros.


## Question 2

In [ ]:
query = "Quais passos devo seguir para defender minha tese ou dissertação?"
query_engine = index.as_query_engine()
answer = query_engine.query(query)

print("answer was:", answer)

answer was: Para defender sua tese ou dissertação, você deve seguir os seguintes passos:

1. Certifique-se de ter cumprido todos os requisitos do Programa de Pós-Graduação.
2. Verifique se todas as coorientações estão devidamente registradas no sistema SIGA.
3. Solicite a verificação completa de sua situação acadêmica enviando um e-mail para a Secretaria de Pós-Graduação.
4. Preencha o Formulário de Publicação (para alunos de Doutorado) ou o Formulário de Formato Alternativo (para alunos que apresentem o trabalho no formato alternativo) e entregue na Secretaria de Pós-Graduação.
5. Envie o Relatório de Verificação de Escrita Original conforme as orientações da Instrução Normativa da CPG 01/2021.
6. Verifique os prazos de solicitação de defesa e preencha as informações da defesa nos formulários online.
7. Suba um arquivo PDF com o texto da dissertação ou tese no sistema.
8. Durante a defesa, os membros da Comissão Examinadora que participarão de forma remota receberão orientações sobre 

# Questão 3

In [ ]:
query = "O orientador pode falar algo durante a apresentação?"
query_engine = index.as_query_engine()
answer = query_engine.query(query)

print(answer.get_formatted_sources())
print("query was:", query)
print("answer was:", answer)

> Source (Doc id: ae395a17-7945-41a4-b62d-555b10ca800c): A partir de 01/05/2022 as defesas devem ocorrer de forma presencial, porém recomenda-se que a Com...

> Source (Doc id: 4974aff1-f9c5-4e19-bd03-f0f6219e1a74): Prazo mínimo de solicitação
Deve solicitar religamento?
Mestrado
Aluno ativo
45 dias
Não
Aluno de...
query was: O orientador pode falar algo durante a apresentação?
answer was: Yes, the advisor can speak during the presentation.
